In [2]:
import sys
import os
sys.path.append('../src')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from utils import save_and_adjust_column_widths


# Read the two files
Ralf_list = pd.read_excel('../data/2ES_targetlist_2025_Apr_suited.xlsx', 
                      dtype={'source_id': str, 'source_id_dr2': str, 'source_id_dr3': str, 'HIP Number': str})

Target_list = pd.read_excel('../results/Gaia_homogeneous_target_selection_2025.09.08_100_granulation.xlsx', 
                      dtype={'source_id': str, 'source_id_dr2': str, 'source_id_dr3': str, 'HIP Number': str})

In [3]:
Ralf_list = Ralf_list[Ralf_list['prio'] == 1]

In [4]:
save_and_adjust_column_widths(Ralf_list, '../results/Ralf_list_prio1.xlsx')

Results saved to ../results/Ralf_list_prio1.xlsx


In [5]:
Target_list['HD Number'] = Target_list['HD Number'].astype(str).str.replace(r'^HD\s+(\d+)$', r'HD\1', regex=True)
Target_list['GJ Number'] = Target_list['GJ Number'].astype(str).str.replace(r'^GJ\s+(\d+)$', r'GJ\1', regex=True)


In [6]:
# Crossmatch Ralf_list with Target_list using "star_ID" and append Target_list info

# First, create a copy to avoid SettingWithCopyWarning
Ralf_list_matched = Ralf_list.copy()

# Prepare a column to store the index of the matched row in Target_list
matched_indices = []

# Iterate through Ralf_list to find matches in Target_list
for idx, row in Ralf_list_matched.iterrows():
    star_id = str(row['star_ID  '])
    if star_id.startswith('HD'):
        match = Target_list[Target_list['HD Number'] == star_id]
    elif star_id.startswith('GJ'):
        match = Target_list[Target_list['GJ Number'] == star_id]
    else:
        match = pd.DataFrame()  # No match if not HD or GJ

    if not match.empty:
        # If multiple matches, take the first one
        matched_indices.append(match.index[0])
    else:
        matched_indices.append(np.nan)

# Add the matched indices to Ralf_list_matched
Ralf_list_matched['Target_list_index'] = matched_indices

# Merge the information from Target_list into Ralf_list_matched
# Only append columns from Target_list that are not already in Ralf_list
target_cols_to_add = [col for col in Target_list.columns if col not in Ralf_list_matched.columns]

for col in target_cols_to_add:
    Ralf_list_matched[col] = Ralf_list_matched['Target_list_index'].apply(
        lambda i: Target_list.at[i, col] if pd.notnull(i) else np.nan
    )

# Drop the helper index column
Ralf_list_matched = Ralf_list_matched.drop(columns=['Target_list_index'])

# Now Ralf_list_matched contains the appended information from Target_list

In [7]:
save_and_adjust_column_widths(Ralf_list_matched, '../results/Ralf_list_matched.xlsx') 

Results saved to ../results/Ralf_list_matched.xlsx


In [11]:
Ralf_list_matched_high_score = Ralf_list_matched[
    (Ralf_list_matched['sum_score'] > 0) & 
    (Ralf_list_matched['activity_noise_floor [m/s]'] < 3)
]
save_and_adjust_column_widths(Ralf_list_matched_high_score, '../results/Ralf_list_matched_high_score.xlsx') 

Results saved to ../results/Ralf_list_matched_high_score.xlsx


In [12]:
Ralf_list_matched_high_score = Ralf_list_matched[
    (Ralf_list_matched['sum_score'] == 0) & 
    (Ralf_list_matched['activity_noise_floor [m/s]'] < 3)
]
save_and_adjust_column_widths(Ralf_list_matched_high_score, '../results/Ralf_list_matched_0_score.xlsx') 

Results saved to ../results/Ralf_list_matched_0_score.xlsx
